# Выявление токсичные комментарии

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучаем модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройм модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Чтение данных
df =  pd.read_csv(r'\Users\Asus\Documents\Python Scripts\Яндекс практикум\Машиное обучение для текстов\toxictweets.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
df.shape

(159571, 2)

In [4]:
# Проверка информаций данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
# Загляним на порцентное соотношение токсичних коментарий
toxic_ratio = pd.Series(df['toxic']==1).sum()/df.shape[0]

print('toxic ratio comments {:.2%}'.format(toxic_ratio))

toxic ratio comments 10.17%


In [6]:
from nltk.corpus import wordnet 

In [18]:

from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
import nltk
import re

In [19]:
from nltk.tokenize import sent_tokenize, word_tokenize
import string
from nltk.corpus import stopwords as nltk_stopwords
# Определенте стоп слов
stop_words = set(nltk_stopwords.words('english'))

#Определение стоварь Знаков препинания 
punctuation = string.punctuation 

#Определение функций ламматизатора
wordnet_lemmatizer = WordNetLemmatizer()


In [20]:
#Функцция лемматизаций
def tokenizer(text):
    #Определение слова токенов
    tokens = [ word for sent in sent_tokenize(text) for word in word_tokenize(sent)]
    tokens = list(filter(lambda t: t not in punctuation, tokens)) # Уборка знаков припенания внутри слов
    tokens = list(filter(lambda t: t.lower() not in stop_words, tokens)) #Уборка стоа-слов
    filtered_tokens = []
    for token in tokens: # Регулярные выражений
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    filtered_tokens = list(
        map(lambda token: wordnet_lemmatizer.lemmatize(token.lower()), filtered_tokens)) #Лемматизаци токенов
    filtered_tokens = list(filter(lambda t: t not in punctuation, filtered_tokens)) # Фильтр пл знаков припенания
    return ' '.join(filtered_tokens)
    

In [21]:
%%time
df['text_clean'] = df['text'].map(tokenizer) #Применение лематизацик тексту


Wall time: 6min 50s


In [22]:
df.shape

(159571, 3)

In [1]:
from sklearn.model_selection import train_test_split

# Определяем функцию для того чтобы разделить датасет на : обущающий, валидацилный и тестовый

def train_test_valid_split(dataframe, test_size, validate_size):
    # Делим на обучующую и тестовую часть
    df_train, df_test = train_test_split(
        dataframe,
        test_size=test_size,
        shuffle = False,
        
    )
    # Воторой раз делим датасет
    post_split_validate_size = validate_size / (1 - test_size)
    df_train, df_validate = train_test_split(
        df_train,
        test_size=post_split_validate_size,
        shuffle =False ,
        
    )
    return df_train, df_test, df_validate

In [24]:
#Деление датасета на трайн, валид и тест
df_train, df_test, df_validate = train_test_valid_split(df, 0.1, 0.2)

print(df_train.shape,
     df_test.shape, 
     df_validate.shape)

(111698, 3) (15958, 3) (31915, 3)


In [25]:
df.head()

,text,toxic,text_clean
0,Explanation\nWhy the edits made under my usern...,0,explanation edits made username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,d'aww match background colour 'm seemingly stu...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man 'm really trying edit war 's guy const...
3,"""\nMore\nI can't make any real suggestions on ...",0,ca n't make real suggestion improvement wonder...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page 's


### Вывод

При подготовке данных перед обучением моделей , при использование tag_pos время исполнения лематизаций свыше 30 мин. Для оптимизаций алгоритма лучше результат получился с применением map. В среднем 10% от коментарий являются токсичными. 

# 2. Обучение

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords

train_corpus = df_train['text_clean'].values.astype('U')
# Определение стоп-слова
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
# Определение TF-IDF для обучения моделей как признаки
tfidf_train = count_tf_idf.fit_transform(train_corpus)

In [27]:
# Определение таргета для обучения моделей
train_labels = df_train['toxic']

In [28]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression() # Обучение модели логистической  регрессий с дкфолтными параметрами
model.fit(tfidf_train, train_labels)
 
validate_labels = df_validate['toxic']
validate_corpus = df_validate['text_clean'].values.astype('U')
tfidf_validate = count_tf_idf.transform(validate_corpus)
 
pred_validate = model.predict(tfidf_validate)


C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [29]:
from sklearn.metrics import f1_score

f1_LR = f1_score(validate_labels ,  pred_validate)

f1_LR

0.7296268088347296

### Вывод

Модель LinearRegression без тюнинга параметров f1 полчилось 0,73, близко к заданому 0,75

In [30]:
%%time
from itertools import product

# подбор параметров для RandomForestRegressor  и вывод RMSE на тестовый датасет 
buf_2 = dict()
for i in product([{'penalty':'l1'},{'penalty':'l2'}],
              [{'max_iter':2},{ 'max_iter':5}, {'max_iter':100}]
              ):
    rfr = LogisticRegression(random_state = 123,**i[0],**i[1])
    rfr.fit(tfidf_train, train_labels)
    pred_2 = rfr.predict(tfidf_validate)
    buf_2[str(i)] = f1_score(validate_labels,pred_2)
buf_2

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Wall time: 9.03 s


{"({'penalty': 'l1'}, {'max_iter': 2})": 0.7538864628820962,
 "({'penalty': 'l1'}, {'max_iter': 5})": 0.7780734433209153,
 "({'penalty': 'l1'}, {'max_iter': 100})": 0.7778566359119944,
 "({'penalty': 'l2'}, {'max_iter': 2})": 0.0,
 "({'penalty': 'l2'}, {'max_iter': 5})": 0.577391304347826,
 "({'penalty': 'l2'}, {'max_iter': 100})": 0.7296268088347296}

In [31]:
%%time

from sklearn.tree import DecisionTreeClassifier
# подбор параметров для RandomForestRegressor  и вывод f1 на тестовый датасет 
buf = dict()
for i in product([{'criterion':'gini'},{'criterion':'entropy'}],
              [{'max_depth':5},{ 'max_depth':10}, {'max_depth':100}]
              ):
    dtc = DecisionTreeClassifier(random_state = 123,**i[0],**i[1])
    dtc.fit(tfidf_train, train_labels)
    pred_3 = dtc.predict(tfidf_validate)
    buf[str(i)] = f1_score(validate_labels,pred_3)
buf

Wall time: 4min 52s


{"({'criterion': 'gini'}, {'max_depth': 5})": 0.5072926162260711,
 "({'criterion': 'gini'}, {'max_depth': 10})": 0.5875682486350272,
 "({'criterion': 'gini'}, {'max_depth': 100})": 0.7164230438521065,
 "({'criterion': 'entropy'}, {'max_depth': 5})": 0.506721348826612,
 "({'criterion': 'entropy'}, {'max_depth': 10})": 0.5786587937579081,
 "({'criterion': 'entropy'}, {'max_depth': 100})": 0.6877615938389419}

In [32]:
%%time
from sklearn.svm import LinearSVC

# подбор параметров для RandomForestRegressor  и вывод f1 на тестовый датасет 
buf_s= dict()
for i in product([{'C':0.1},{'C':1.0} ,{'C':2.5}],
              [{'max_iter':5},{ 'max_iter':10}, {'max_iter':100}]
              ):
    svc = LinearSVC(random_state = 123,**i[0],**i[1])
    svc.fit(tfidf_train, train_labels)
    pred_3 = svc.predict(tfidf_validate)
    buf_s[str(i)] = f1_score(validate_labels,pred_3)
buf_s

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Wall time: 9.23 s


{"({'C': 0.1}, {'max_iter': 5})": 0.7530020321448365,
 "({'C': 0.1}, {'max_iter': 10})": 0.7340182648401826,
 "({'C': 0.1}, {'max_iter': 100})": 0.7343601445141662,
 "({'C': 1.0}, {'max_iter': 5})": 0.7932324711241256,
 "({'C': 1.0}, {'max_iter': 10})": 0.7823969385980171,
 "({'C': 1.0}, {'max_iter': 100})": 0.7834693523181108,
 "({'C': 2.5}, {'max_iter': 5})": 0.7861707357037522,
 "({'C': 2.5}, {'max_iter': 10})": 0.7804149377593361,
 "({'C': 2.5}, {'max_iter': 100})": 0.7784004075394803}

In [34]:
# Поиск найлучше рузкльтата по RMSE для кажлой модели 
best_result = []
for buffer in (buf, buf_2, buf_s):
    param = max(buffer.items(), key=lambda x: x[1]) 
    best_result.append(param)

best_result

[("({'criterion': 'gini'}, {'max_depth': 100})", 0.7164230438521065),
 ("({'penalty': 'l1'}, {'max_iter': 5})", 0.7780734433209153),
 ("({'C': 1.0}, {'max_iter': 5})", 0.7932324711241256)]

# 3. Выводы

In [40]:
test_labels = df_test['toxic'] # Определение таргетов для теста
test_corpus = df_test['text_clean'].values.astype('U') # Определение признаком для теста

tfidf_test = count_tf_idf.transform(test_corpus) 

best_model = LinearSVC(random_state = 123 ,  max_iter= 5) # Модель с лучшими параметрами на валид

best_model.fit(tfidf_train, train_labels) # Обучение лучше модель
pred_test = best_model.predict(tfidf_test) # Предсказание на тестовой выборке


f1_best = f1_score(test_labels ,  pred_test) # Метрика F1 на тестовой выборке

f1_best


0.7871794871794872

### Вывод 

Лучше значение метррики F1 проучается при модели LogisticRegression с параметрами 'penalty'= 'l1' и 'max_iter'= 5, что выше заданому по заданию